In [1]:
import time, os
import torch
import numpy as np
import torch.nn as nn

import pandas as pd
from tqdm import tqdm

import json
import pickle

/users/cost/jlovonme/miniconda3/envs/hierarchy/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

In [4]:
# Dataset sizes
_dataset_base = "/users/cost/jlovonme/data"
ds_sizes = {}
for prop in range(9):
    _propfile = "hierarchy_props/bansal_with_defs_testprop_{prop}.json".format(prop=str(prop))
    df = pd.read_json(load_json(os.path.join(_dataset_base,_propfile)), orient="index")
    ndf = df[df['valid']==1]
    ds_sizes[prop] = len(ndf)
    print(prop, len(ndf))

0 1435
1 7658
2 8273
3 6522
4 7658
5 8273
6 6522
7 33160
8 31746


# Gather results obtained after parallel launch with gather_results.py script

In [3]:
def create_table_results(itype, models,props=9 ):
    bigtable = []
    _props = 9
    for _prop in range(props):
        for _model in _models:
            _ifile = os.path.join(basepath,'results_{input_type}_p{prop}_{model}.json'.format(input_type=itype,
                                                                                                   prop=_prop,
                                                                                                   model=_model.replace('/','_')))
            df = pd.read_json(load_json(_ifile), orient="index")
            bigtable.append(df)
    bigdf = pd.concat(bigtable)
    return bigdf


def get_last_layers(bigdf):

    # Filter only last layer for each model
    maxlayer_dict = {}
    for (idx,row) in bigdf.groupby(['model']).max()['layer'].iteritems():
        maxlayer_dict[idx] = row


    last_layers = []
    for mod,lay in maxlayer_dict.items():
        last_layers.append(bigdf[(bigdf['model']==mod) & (bigdf['layer']==lay)])

    lastlayer_df = pd.concat(last_layers)
    return lastlayer_df


def get_top_by_prop(df,prop, method):
    pdf =df[df['prop'] == prop]
    lmethod = [method,method+"_l1",method+"_l2",method+"_l3"]
    if method=='avg':
        lmethod[0] = 'avg_a'
        method='avg_a'
    return pdf.sort_values(method, ascending=False).groupby(['model']).head(1).sort_values(method,ascending=False)[['input','model','prop','layer',*lmethod]]

def get_last_by_prop(df,prop):
    pdf =df[df['prop'] == prop]
    #return pdf.sort_values(method,ascending=False)[['input','model','prop','layer',method]]
    return pdf

In [6]:
basepath = './computed_results_v2'
_dataset_base = "/users/cost/jlovonme/data"
_inputtype = 'name' # name or concept
#_methods = ['cls','avg_all']
_methods = ['cls','avg_a','cls_l1','avg_l1','cls_l2','avg_l2','cls_l3','avg_l3','ecls','eavg','ecls_l1','eavg_l1','ecls_l2','eavg_l2','ecls_l3','eavg_l3']
_models = ['ft-cc','ft-wiki','bert-base-cased','bert-large-cased','roberta-base','roberta-large','sentence-transformers/all-mpnet-base-v2',
          'allenai/scibert_scivocab_cased','ctp','ernie-base','sentence-transformers/bert-base-nli-mean-tokens','sentence-transformers/all-distilroberta-v1']
#_models = ['bert-base-cased']
orderedmethod = ['cls','cls_l1','cls_l2','cls_l3','avg_a','avg_l1','avg_l2','avg_l3','ecls','ecls_l1','ecls_l2','ecls_l3','eavg','eavg_l1','eavg_l2','eavg_l3']

# For name

In [8]:
_bigdf = create_table_results(_inputtype, _models,props=9 )

In [9]:
_lastlayers = get_last_layers(_bigdf)

In [24]:
for p in range(9):
    for method in ['cls','avg','ecls','eavg']:
        print("Property",p, method)
        display(get_top_by_prop(_bigdf,p, method))

Property 0 cls


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3
0,name,ft-cc,0,0,72.61,64.18,68.43,66.34
0,name,ft-wiki,0,0,70.73,58.19,70.10,68.15
6,name,sentence-transformers/all-distilroberta-v1,0,6,68.85,57.84,69.20,68.36
2,name,sentence-transformers/bert-base-nli-mean-tokens,0,2,67.32,55.68,66.41,65.23
4,name,allenai/scibert_scivocab_cased,0,4,67.25,55.96,66.20,65.30
5,name,ernie-base,0,5,66.62,56.31,66.69,65.92
1,name,bert-base-cased,0,1,64.81,57.28,65.16,62.44
12,name,sentence-transformers/all-mpnet-base-v2,0,12,64.74,66.55,65.37,65.57
17,name,ctp,0,17,63.97,55.33,63.97,63.00
1,name,roberta-base,0,1,61.81,54.77,59.09,56.93


Property 0 avg


,input,model,prop,layer,avg_a,avg_l1,avg_l2,avg_l3
0,name,ft-cc,0,0,72.61,64.18,68.43,66.34
0,name,ft-wiki,0,0,70.73,58.19,70.10,68.15
8,name,ctp,0,8,70.31,55.33,68.92,66.76
6,name,sentence-transformers/all-distilroberta-v1,0,6,70.31,55.82,70.31,69.48
8,name,roberta-large,0,8,70.24,55.61,67.67,64.67
2,name,sentence-transformers/all-mpnet-base-v2,0,2,69.62,55.33,68.71,67.94
3,name,bert-base-cased,0,3,69.41,53.80,65.85,64.39
4,name,allenai/scibert_scivocab_cased,0,4,69.34,56.10,68.36,66.20
6,name,bert-large-cased,0,6,67.60,50.52,63.69,61.81
6,name,sentence-transformers/bert-base-nli-mean-tokens,0,6,67.25,52.06,64.53,63.28


Property 0 ecls


...

,input,model,prop,layer,avg_a,avg_l1,avg_l2,avg_l3
0,name,ft-cc,8,0,69.29,61.84,64.67,63.14
6,name,sentence-transformers/all-distilroberta-v1,8,6,67.64,53.56,67.10,66.15
1,name,bert-large-cased,8,1,67.34,54.93,65.84,64.42
0,name,ft-wiki,8,0,67.09,56.34,64.79,63.39
1,name,bert-base-cased,8,1,66.59,54.97,65.21,63.99
10,name,ctp,8,10,66.17,52.47,64.87,63.59
1,name,allenai/scibert_scivocab_cased,8,1,65.70,56.53,66.04,65.13
6,name,roberta-large,8,6,64.99,53.26,64.04,62.59
0,name,roberta-base,8,0,64.98,56.54,64.10,62.79
12,name,sentence-transformers/all-mpnet-base-v2,8,12,63.02,60.50,63.10,62.81


Property 8 ecls


,input,model,prop,layer,ecls,ecls_l1,ecls_l2,ecls_l3
6,name,sentence-transformers/all-distilroberta-v1,8,6,66.29,66.24,66.29,66.28
0,name,ft-wiki,8,0,64.91,64.75,64.89,64.87
11,name,bert-base-cased,8,11,64.53,64.56,64.53,64.52
0,name,ft-cc,8,0,64.32,63.53,64.18,63.59
12,name,sentence-transformers/all-mpnet-base-v2,8,12,62.79,62.65,62.77,62.77
2,name,allenai/scibert_scivocab_cased,8,2,62.45,62.53,62.46,62.46
8,name,roberta-large,8,8,61.72,61.70,61.71,61.71
11,name,roberta-base,8,11,61.23,61.24,61.24,61.25
12,name,bert-large-cased,8,12,61.21,61.20,61.22,61.21
8,name,ctp,8,8,60.50,60.49,60.50,60.50


Property 8 eavg


,input,model,prop,layer,eavg,eavg_l1,eavg_l2,eavg_l3
6,name,sentence-transformers/all-distilroberta-v1,8,6,67.42,67.36,67.41,67.42
1,name,bert-large-cased,8,1,66.40,66.34,66.40,66.39
1,name,bert-base-cased,8,1,66.15,66.11,66.15,66.15
10,name,ctp,8,10,65.90,65.89,65.89,65.90
0,name,ft-wiki,8,0,64.91,64.75,64.89,64.87
10,name,roberta-large,8,10,64.78,64.75,64.78,64.78
3,name,roberta-base,8,3,64.63,64.56,64.62,64.62
2,name,allenai/scibert_scivocab_cased,8,2,64.42,64.36,64.41,64.41
0,name,ft-cc,8,0,64.32,63.53,64.18,63.59
12,name,sentence-transformers/all-mpnet-base-v2,8,12,63.09,63.03,63.09,63.10


In [7]:
for p in range(9):
    #for method in _methods:
    print("Property",p)
    display(get_last_by_prop(_lastlayers,p)[['input','model','prop','layer',*orderedmethod]])

Property 0


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
12,name,allenai/scibert_scivocab_cased,0,12,61.53,56.79,61.53,61.39,65.57,60.42,66.13,66.69,61.60,61.67,61.67,61.67,66.97,67.18,66.97,66.97
12,name,bert-base-cased,0,12,61.32,51.57,59.30,57.42,54.98,50.31,52.75,52.68,61.46,62.09,61.53,61.53,53.87,53.59,53.94,53.94
24,name,bert-large-cased,0,24,51.99,51.85,53.17,53.73,61.46,48.85,56.59,54.43,52.20,52.33,52.20,52.20,53.52,53.10,53.52,53.52
24,name,ctp,0,24,58.95,59.30,58.95,58.89,58.12,56.72,57.70,57.49,59.02,59.02,59.02,59.02,58.40,58.33,58.40,58.40
12,name,ernie-base,0,12,61.67,53.10,59.58,57.98,66.34,56.03,64.04,62.65,61.53,61.46,61.53,61.60,66.06,66.20,66.06,66.13
0,name,ft-cc,0,0,72.61,64.18,68.43,66.34,72.61,64.18,68.43,66.34,66.83,65.09,66.27,66.20,66.83,65.09,66.27,66.20
0,name,ft-wiki,0,0,70.73,58.19,70.10,68.15,70.73,58.19,70.10,68.15,70.10,69.62,69.97,69.83,70.10,69.62,69.97,69.83
12,name,roberta-base,0,12,58.12,53.31,54.01,53.80,57.77,54.22,55.54,55.26,57.70,56.93,57.77,57.56,57.98,58.05,57.91,57.77
24,name,roberta-large,0,24,61.53,55.47,61.60,61.67,62.44,55.19,60.70,58.95,61.53,61.53,61.53,61.53,62.30,62.30,62.30,62.30
6,name,sentence-transformers/all-distilroberta-v1,0,6,68.85,57.84,69.20,68.36,70.31,55.82,70.31,69.48,68.43,68.50,68.43,68.50,70.10,69.97,70.10,70.17


Property 1


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
12,name,allenai/scibert_scivocab_cased,1,12,38.76,40.02,38.27,38.12,48.67,44.08,47.61,47.11,38.16,38.16,38.17,38.17,42.57,42.50,42.57,42.57
12,name,bert-base-cased,1,12,47.60,44.40,46.60,45.76,35.02,26.16,30.49,29.43,46.85,46.70,46.84,46.84,29.58,29.32,29.56,29.55
24,name,bert-large-cased,1,24,33.13,40.83,35.28,36.16,52.27,30.80,39.75,36.88,35.57,35.58,35.58,35.58,33.14,32.91,33.15,33.14
24,name,ctp,1,24,42.64,43.67,42.69,42.70,42.22,47.61,42.39,42.65,42.64,42.65,42.64,42.64,42.18,42.18,42.18,42.18
12,name,ernie-base,1,12,48.15,47.11,47.49,47.56,50.09,50.46,50.35,51.38,47.90,47.83,47.92,47.94,48.00,47.96,48.00,48.03
0,name,ft-cc,1,0,66.39,49.28,54.87,52.83,66.39,49.28,54.87,52.83,50.89,49.13,50.93,50.33,50.89,49.13,50.93,50.33
0,name,ft-wiki,1,0,60.79,49.62,57.36,55.29,60.79,49.62,57.36,55.29,53.66,52.31,53.50,53.40,53.66,52.31,53.50,53.40
12,name,roberta-base,1,12,49.50,48.75,48.07,48.73,44.54,46.85,45.99,46.45,49.62,48.76,49.53,49.63,44.97,44.82,45.00,44.99
24,name,roberta-large,1,24,51.21,49.95,51.10,51.31,42.57,45.42,42.19,42.71,51.24,51.20,51.24,51.24,42.23,42.24,42.23,42.22
6,name,sentence-transformers/all-distilroberta-v1,1,6,56.42,50.04,55.47,55.00,58.98,46.75,56.45,55.08,61.44,61.36,61.45,61.45,46.59,46.68,46.59,46.58


Property 2


...

,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
12,name,allenai/scibert_scivocab_cased,6,12,37.96,41.37,37.44,37.43,47.30,41.37,44.80,44.08,36.71,36.72,36.71,36.71,40.79,40.62,40.79,40.79
12,name,bert-base-cased,6,12,51.69,47.07,49.85,49.42,41.64,34.67,37.83,36.84,50.17,50.08,50.17,50.15,37.29,37.07,37.29,37.29
24,name,bert-large-cased,6,24,41.97,46.44,43.24,43.81,54.92,32.57,42.06,39.83,42.16,42.16,42.16,42.16,38.06,37.70,38.04,38.03
24,name,ctp,6,24,39.02,39.87,39.08,39.05,38.42,39.48,38.47,38.39,39.04,39.07,39.04,39.04,38.39,38.39,38.39,38.39
12,name,ernie-base,6,12,51.44,43.38,48.37,46.49,49.49,49.08,50.00,50.86,50.97,50.74,51.01,51.03,48.64,48.70,48.65,48.67
0,name,ft-cc,6,0,60.44,55.12,56.62,56.92,60.44,55.12,56.62,56.92,55.06,55.34,56.15,56.52,55.06,55.34,56.15,56.52
0,name,ft-wiki,6,0,59.80,52.61,58.20,57.48,59.80,52.61,58.20,57.48,52.01,52.04,51.96,51.90,52.01,52.04,51.96,51.90
12,name,roberta-base,6,12,52.25,45.40,45.83,45.52,41.05,48.56,43.13,44.10,51.56,51.18,51.70,51.79,42.00,42.16,42.01,42.04
24,name,roberta-large,6,24,48.39,47.07,48.19,48.14,43.36,41.11,40.36,39.31,48.39,48.36,48.39,48.39,42.85,42.61,42.82,42.78
6,name,sentence-transformers/all-distilroberta-v1,6,6,60.36,46.37,56.53,54.12,60.23,41.83,56.30,53.13,62.54,62.47,62.51,62.53,42.30,42.23,42.33,42.33


Property 7


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
12,name,allenai/scibert_scivocab_cased,7,12,63.74,59.82,63.94,64.00,63.27,59.31,63.24,63.28,63.67,63.70,63.66,63.66,64.41,64.42,64.41,64.41
12,name,bert-base-cased,7,12,64.48,57.39,63.29,62.42,61.58,59.05,62.26,62.21,64.73,64.67,64.72,64.72,62.60,62.65,62.60,62.61
24,name,bert-large-cased,7,24,59.38,53.38,58.30,57.17,60.27,59.48,62.09,62.33,59.08,59.00,59.08,59.08,62.79,62.78,62.80,62.80
24,name,ctp,7,24,58.11,56.34,58.10,58.07,58.73,54.98,58.66,58.56,58.13,58.13,58.13,58.13,58.75,58.75,58.75,58.75
12,name,ernie-base,7,12,59.07,52.48,58.16,57.01,60.47,55.13,60.04,59.67,59.07,58.97,59.05,59.04,60.29,60.28,60.30,60.29
0,name,ft-cc,7,0,67.19,62.18,64.35,63.02,67.19,62.18,64.35,63.02,62.28,62.05,62.01,61.89,62.28,62.05,62.01,61.89
0,name,ft-wiki,7,0,64.91,60.05,64.81,63.97,64.91,60.05,64.81,63.97,64.45,64.63,64.47,64.49,64.45,64.63,64.47,64.49
12,name,roberta-base,7,12,62.58,52.51,54.07,53.23,63.53,54.03,60.93,59.16,62.38,60.05,62.30,62.14,63.58,63.43,63.55,63.57
24,name,roberta-large,7,24,63.31,56.03,63.21,62.99,63.48,51.09,61.07,59.19,63.32,63.31,63.32,63.32,63.51,63.42,63.51,63.52
6,name,sentence-transformers/all-distilroberta-v1,7,6,66.88,56.55,66.25,65.38,66.94,55.68,66.84,66.06,65.73,65.74,65.72,65.72,66.93,66.95,66.93,66.94


Property 8


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
12,name,allenai/scibert_scivocab_cased,8,12,58.31,55.07,58.02,57.92,59.89,54.79,59.46,59.18,58.30,58.34,58.30,58.30,59.86,59.84,59.87,59.87
12,name,bert-base-cased,8,12,63.47,54.59,62.27,61.09,61.80,56.05,61.71,61.05,63.23,63.19,63.25,63.24,61.93,61.92,61.93,61.93
24,name,bert-large-cased,8,24,56.60,53.21,55.80,55.37,59.26,55.69,59.28,58.91,56.29,56.21,56.28,56.28,59.83,59.76,59.84,59.85
24,name,ctp,8,24,54.88,53.69,54.84,54.82,55.06,52.13,55.08,54.97,54.88,54.88,54.88,54.88,55.01,55.04,55.01,55.01
12,name,ernie-base,8,12,57.47,49.62,55.14,53.63,59.24,51.16,58.30,57.27,57.42,57.33,57.40,57.40,59.06,58.97,59.04,59.02
0,name,ft-cc,8,0,69.29,61.84,64.67,63.14,69.29,61.84,64.67,63.14,64.32,63.53,64.18,63.59,64.32,63.53,64.18,63.59
0,name,ft-wiki,8,0,67.09,56.34,64.79,63.39,67.09,56.34,64.79,63.39,64.91,64.75,64.89,64.87,64.91,64.75,64.89,64.87
12,name,roberta-base,8,12,60.42,51.80,53.40,52.57,61.29,51.82,58.23,56.12,60.62,59.64,60.63,60.68,60.72,60.58,60.71,60.71
24,name,roberta-large,8,24,61.52,54.20,61.30,61.11,61.32,51.72,59.95,58.39,61.53,61.53,61.53,61.53,61.33,61.31,61.32,61.32
6,name,sentence-transformers/all-distilroberta-v1,8,6,66.84,55.84,66.00,65.30,67.64,53.56,67.10,66.15,66.29,66.24,66.29,66.28,67.42,67.36,67.41,67.42


# For Concept

In [10]:
_bigdf = create_table_results('concept', _models,props=9 )
_lastlayers = get_last_layers(_bigdf)

In [26]:
for p in range(9):
    for method in ['cls','avg','ecls','eavg']:
        print("Property",p, method)
        display(get_top_by_prop(_bigdf,p, method))

Property 0 cls


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3
12,concept,sentence-transformers/all-mpnet-base-v2,0,12,83.21,79.72,83.00,82.86
6,concept,sentence-transformers/all-distilroberta-v1,0,6,83.14,57.49,80.98,79.37
11,concept,bert-large-cased,0,11,81.25,57.56,80.14,78.05
0,concept,ft-wiki,0,0,79.65,64.95,73.17,69.62
4,concept,sentence-transformers/bert-base-nli-mean-tokens,0,4,78.82,55.40,76.72,74.15
4,concept,ernie-base,0,4,75.26,56.79,74.22,72.13
12,concept,bert-base-cased,0,12,72.61,54.70,67.60,64.74
2,concept,allenai/scibert_scivocab_cased,0,2,71.92,54.98,69.13,66.62
12,concept,roberta-base,0,12,65.99,53.94,56.38,54.91
5,concept,ctp,0,5,63.69,51.92,61.46,58.68


Property 0 avg


,input,model,prop,layer,avg_a,avg_l1,avg_l2,avg_l3
11,concept,sentence-transformers/all-mpnet-base-v2,0,11,86.20,69.48,85.02,83.14
6,concept,sentence-transformers/all-distilroberta-v1,0,6,85.16,57.14,83.28,80.07
1,concept,bert-large-cased,0,1,82.58,65.02,80.49,77.42
1,concept,allenai/scibert_scivocab_cased,0,1,80.42,59.30,76.59,73.38
1,concept,bert-base-cased,0,1,80.00,61.74,77.84,74.91
1,concept,ernie-base,0,1,79.86,63.07,77.77,74.77
0,concept,ft-wiki,0,0,79.65,64.95,73.17,69.62
3,concept,sentence-transformers/bert-base-nli-mean-tokens,0,3,79.30,60.35,76.10,73.31
10,concept,ctp,0,10,77.42,55.05,70.31,65.78
12,concept,roberta-base,0,12,76.38,60.56,69.13,65.23


Property 0 ecls


...

,input,model,prop,layer,avg_a,avg_l1,avg_l2,avg_l3
12,concept,sentence-transformers/all-mpnet-base-v2,8,12,81.42,77.64,81.39,80.90
1,concept,bert-large-cased,8,1,79.78,55.83,76.37,73.14
6,concept,sentence-transformers/all-distilroberta-v1,8,6,79.51,56.68,77.55,75.15
0,concept,ft-wiki,8,0,78.97,63.83,73.85,69.83
1,concept,allenai/scibert_scivocab_cased,8,1,78.75,54.61,74.89,71.15
1,concept,ernie-base,8,1,77.42,56.54,74.06,70.74
1,concept,bert-base-cased,8,1,76.89,56.08,73.20,70.01
1,concept,sentence-transformers/bert-base-nli-mean-tokens,8,1,76.56,57.14,74.04,71.10
1,concept,roberta-large,8,1,75.87,53.66,72.75,69.03
1,concept,ctp,8,1,75.80,53.88,73.04,69.12


Property 8 ecls


,input,model,prop,layer,ecls,ecls_l1,ecls_l2,ecls_l3
12,concept,sentence-transformers/all-mpnet-base-v2,8,12,78.31,78.27,78.30,78.29
10,concept,bert-large-cased,8,10,76.60,76.72,76.60,76.59
6,concept,sentence-transformers/all-distilroberta-v1,8,6,76.41,76.52,76.41,76.42
0,concept,ft-wiki,8,0,75.67,73.89,75.33,75.03
4,concept,sentence-transformers/bert-base-nli-mean-tokens,8,4,75.26,75.28,75.26,75.26
12,concept,ernie-base,8,12,73.13,73.00,73.15,73.16
12,concept,roberta-base,8,12,71.18,68.31,71.12,71.08
12,concept,bert-base-cased,8,12,70.88,70.85,70.87,70.87
2,concept,allenai/scibert_scivocab_cased,8,2,69.43,69.32,69.42,69.44
5,concept,ctp,8,5,63.42,63.27,63.42,63.42


Property 8 eavg


,input,model,prop,layer,eavg,eavg_l1,eavg_l2,eavg_l3
12,concept,sentence-transformers/all-mpnet-base-v2,8,12,80.58,80.66,80.59,80.60
2,concept,bert-large-cased,8,2,77.44,77.25,77.43,77.42
5,concept,sentence-transformers/all-distilroberta-v1,8,5,76.69,76.45,76.68,76.68
1,concept,ctp,8,1,75.97,75.82,75.98,75.97
1,concept,roberta-large,8,1,75.92,75.83,75.91,75.90
0,concept,ft-wiki,8,0,75.67,73.89,75.33,75.03
1,concept,allenai/scibert_scivocab_cased,8,1,75.17,75.01,75.17,75.16
12,concept,ernie-base,8,12,74.97,74.82,74.95,74.96
1,concept,bert-base-cased,8,1,74.43,74.28,74.43,74.43
2,concept,sentence-transformers/bert-base-nli-mean-tokens,8,2,74.37,74.12,74.37,74.37


In [29]:
for p in range(9):
    #for method in _methods:
    print("Property",p)
    display(get_last_by_prop(_lastlayers,p)[['input','model','prop','layer',*orderedmethod]])

Property 0


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
12,concept,allenai/scibert_scivocab_cased,0,12,65.09,55.75,65.02,64.18,67.46,56.86,64.39,62.72,65.16,65.09,65.16,65.16,68.99,68.78,68.99,68.99
12,concept,bert-base-cased,0,12,72.61,54.70,67.60,64.74,76.03,59.37,74.29,70.87,72.40,71.85,72.40,72.54,75.68,75.68,75.68,75.68
24,concept,bert-large-cased,0,24,68.15,51.99,66.20,62.72,78.89,57.00,77.14,76.17,68.71,68.50,68.71,68.71,77.98,78.05,77.91,77.91
24,concept,ctp,0,24,60.07,58.05,60.14,60.07,60.07,57.98,60.42,60.49,60.07,60.07,60.07,60.07,60.70,60.70,60.70,60.77
12,concept,ernie-base,0,12,68.99,54.91,65.30,61.67,78.05,56.86,74.43,70.31,69.20,68.78,69.06,69.06,77.77,77.63,77.77,77.77
0,concept,ft-cc,0,0,63.34,56.79,58.68,56.24,63.34,56.79,58.68,56.24,60.35,58.82,59.51,58.75,60.35,58.82,59.51,58.75
0,concept,ft-wiki,0,0,79.65,64.95,73.17,69.62,79.65,64.95,73.17,69.62,77.00,75.33,76.66,76.31,77.00,75.33,76.66,76.31
12,concept,roberta-base,0,12,65.99,53.94,56.38,54.91,76.38,60.56,69.13,65.23,66.27,64.60,66.13,66.20,76.59,75.82,76.52,76.59
24,concept,roberta-large,0,24,58.54,55.26,58.12,58.05,75.05,47.74,61.46,55.05,58.54,58.54,58.54,58.54,74.84,74.43,74.84,74.84
6,concept,sentence-transformers/all-distilroberta-v1,0,6,83.14,57.49,80.98,79.37,85.16,57.14,83.28,80.07,79.09,79.23,79.09,79.09,80.63,80.49,80.63,80.70


Property 1


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
12,concept,allenai/scibert_scivocab_cased,1,12,38.80,42.94,38.80,38.84,40.43,44.89,40.89,40.90,38.77,38.80,38.78,38.78,40.09,40.15,40.08,40.10
12,concept,bert-base-cased,1,12,42.86,48.54,45.26,45.83,38.18,43.82,37.63,38.47,42.79,42.87,42.80,42.80,36.51,36.55,36.48,36.52
24,concept,bert-large-cased,1,24,44.16,46.85,44.96,45.22,41.54,44.62,40.96,41.59,44.48,44.38,44.48,44.48,38.50,38.27,38.47,38.46
24,concept,ctp,1,24,22.88,23.34,22.93,22.92,22.34,21.91,21.90,21.48,22.92,22.92,22.92,22.92,22.64,22.66,22.64,22.64
12,concept,ernie-base,1,12,37.16,39.84,37.28,37.71,43.00,40.45,41.84,41.20,37.16,36.84,37.12,37.12,42.40,42.30,42.40,42.33
0,concept,ft-cc,1,0,42.91,43.01,42.48,42.60,42.91,43.01,42.48,42.60,43.28,43.08,43.21,42.92,43.28,43.08,43.21,42.92
0,concept,ft-wiki,1,0,49.93,48.98,48.88,48.67,49.93,48.98,48.88,48.67,49.52,49.39,49.36,49.41,49.52,49.39,49.36,49.41
12,concept,roberta-base,1,12,39.07,51.65,49.40,50.29,38.38,42.33,38.78,40.42,39.45,41.93,39.61,39.80,35.78,35.65,35.79,35.81
24,concept,roberta-large,1,24,40.94,44.72,40.94,41.09,37.09,40.13,38.22,38.65,40.95,40.96,40.95,40.95,37.26,37.22,37.28,37.27
6,concept,sentence-transformers/all-distilroberta-v1,1,6,68.62,48.02,66.45,63.40,68.18,44.05,63.01,58.71,65.90,65.81,65.92,65.92,69.54,68.79,69.48,69.43


Property 2


...

,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
12,concept,allenai/scibert_scivocab_cased,6,12,45.72,45.48,45.52,45.54,41.70,44.42,43.81,43.88,45.74,45.65,45.74,45.74,39.68,39.85,39.67,39.65
12,concept,bert-base-cased,6,12,48.54,51.69,48.68,49.91,44.19,41.52,41.95,42.20,47.76,47.76,47.73,47.72,41.03,40.72,41.05,41.06
24,concept,bert-large-cased,6,24,49.37,49.54,48.54,48.08,50.48,46.38,49.03,48.10,49.79,49.75,49.79,49.77,45.86,45.66,45.91,45.91
24,concept,ctp,6,24,21.79,21.82,21.77,21.85,21.34,21.51,21.30,21.28,21.79,21.79,21.79,21.79,21.48,21.48,21.48,21.48
12,concept,ernie-base,6,12,49.66,48.76,49.48,48.83,47.38,43.67,45.28,44.89,49.60,49.60,49.62,49.62,44.33,43.96,44.25,44.25
0,concept,ft-cc,6,0,48.54,47.99,48.76,48.18,48.54,47.99,48.76,48.18,48.64,48.53,49.05,48.70,48.64,48.53,49.05,48.70
0,concept,ft-wiki,6,0,51.01,44.82,48.33,46.60,51.01,44.82,48.33,46.60,46.66,45.78,46.52,46.55,46.66,45.78,46.52,46.55
12,concept,roberta-base,6,12,55.43,49.23,50.00,49.48,46.70,34.56,36.74,35.80,54.71,52.88,54.32,54.05,39.70,38.72,39.62,39.56
24,concept,roberta-large,6,24,50.48,48.22,50.26,50.08,37.78,45.95,41.57,43.47,50.52,50.51,50.52,50.52,37.78,38.01,37.73,37.72
6,concept,sentence-transformers/all-distilroberta-v1,6,6,66.16,49.77,63.75,61.79,68.17,46.24,62.50,59.71,62.57,62.70,62.59,62.60,62.27,62.28,62.30,62.30


Property 7


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
12,concept,allenai/scibert_scivocab_cased,7,12,68.82,57.28,68.42,67.97,68.70,55.96,66.11,64.09,68.77,68.74,68.77,68.77,68.60,68.51,68.60,68.59
12,concept,bert-base-cased,7,12,72.79,55.60,69.71,66.75,75.73,59.85,74.27,72.06,72.75,72.76,72.76,72.75,75.68,75.60,75.67,75.66
24,concept,bert-large-cased,7,24,68.13,54.50,66.60,64.94,76.12,58.00,75.37,73.74,68.21,68.15,68.21,68.21,76.61,76.60,76.61,76.61
24,concept,ctp,7,24,63.90,62.94,63.88,63.88,65.05,63.23,65.08,65.15,63.88,63.88,63.88,63.88,64.95,64.96,64.95,64.95
12,concept,ernie-base,7,12,72.21,55.32,68.50,65.00,75.52,58.58,73.44,71.08,72.26,72.12,72.26,72.25,74.93,74.65,74.93,74.92
0,concept,ft-cc,7,0,64.37,59.41,61.16,59.52,64.37,59.41,61.16,59.52,63.73,60.80,61.95,60.97,63.73,60.80,61.95,60.97
0,concept,ft-wiki,7,0,77.38,64.06,72.62,69.00,77.38,64.06,72.62,69.00,75.75,73.86,75.30,74.94,75.75,73.86,75.30,74.94
12,concept,roberta-base,7,12,72.03,55.14,58.80,56.98,75.60,57.95,66.76,63.24,71.84,69.32,71.79,71.76,75.64,74.89,75.62,75.61
24,concept,roberta-large,7,24,63.97,57.19,63.88,63.75,75.27,54.86,66.24,61.80,63.98,63.97,63.98,63.98,75.26,75.02,75.25,75.25
6,concept,sentence-transformers/all-distilroberta-v1,7,6,78.00,61.78,78.30,77.40,77.75,59.15,77.05,74.78,75.33,75.41,75.33,75.33,72.96,73.07,72.96,72.95


Property 8


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
12,concept,allenai/scibert_scivocab_cased,8,12,66.21,57.28,66.00,65.60,66.38,54.18,63.73,61.98,66.17,66.18,66.17,66.17,65.49,65.36,65.49,65.48
12,concept,bert-base-cased,8,12,70.82,56.26,68.79,65.84,73.56,56.18,70.94,68.54,70.88,70.85,70.87,70.87,73.05,72.82,73.05,73.05
24,concept,bert-large-cased,8,24,67.09,54.11,65.03,63.34,75.92,55.52,74.21,71.74,67.08,66.99,67.08,67.07,75.51,75.40,75.52,75.50
24,concept,ctp,8,24,56.03,55.27,55.98,55.98,57.05,54.70,56.90,56.84,56.03,56.01,56.03,56.03,57.05,57.05,57.05,57.05
12,concept,ernie-base,8,12,73.05,56.46,69.55,66.17,75.68,56.44,73.01,70.09,73.13,73.00,73.15,73.16,74.97,74.82,74.95,74.96
0,concept,ft-cc,8,0,63.35,57.28,59.04,57.37,63.35,57.28,59.04,57.37,61.88,58.44,59.83,58.61,61.88,58.44,59.83,58.61
0,concept,ft-wiki,8,0,78.97,63.83,73.85,69.83,78.97,63.83,73.85,69.83,75.67,73.89,75.33,75.03,75.67,73.89,75.33,75.03
12,concept,roberta-base,8,12,71.57,55.23,58.18,56.69,73.97,53.40,62.61,58.93,71.18,68.31,71.12,71.08,72.81,71.67,72.80,72.73
24,concept,roberta-large,8,24,62.91,56.64,62.81,62.66,73.17,54.12,65.60,61.13,62.92,62.91,62.92,62.92,73.29,73.25,73.28,73.29
6,concept,sentence-transformers/all-distilroberta-v1,8,6,79.20,57.58,77.99,76.73,79.51,56.68,77.55,75.15,76.41,76.52,76.41,76.42,73.91,73.78,73.92,73.91


In [11]:
# table to show in Latex: We study only property 1 because similar trends in others\
# Also main works focus on hypernym, so we study representations on property 1 that fit best to this type of evaluation, to be fair with enhanced models
# We use the concept input (w/ definitions) because they are contextualized models
# We evalaute cosinus distance, since euclidean gives similar trends, and it's a lot of space
cur_df = get_last_by_prop(_lastlayers,0)[['model',*orderedmethod[:-8]]]
for m in orderedmethod[:-8]:
    cur_df[m]=cur_df[m].round(decimals=1)
cur_df['model'] = ['Scibert','BERT','BERT-L','CTP','ERNIE','FT-cc','FT-wiki','RoBERTa','RoBERTa-L','DistilRoB','MPNet','S-BERT']
cur_df = cur_df.set_index('model')
cur_df = cur_df.transpose()[['BERT','BERT-L','RoBERTa','RoBERTa-L','S-BERT','DistilRoB','MPNet','ERNIE','CTP']]
#cur_df = cur_df.transpose()[]
print(cur_df.to_latex(index=True))

\begin{tabular}{lrrrrrrrrr}
\toprule
model &  BERT &  BERT-L &  RoBERTa &  RoBERTa-L &  S-BERT &  DistilRoB &  MPNet &  ERNIE &   CTP \\
\midrule
cls    &  72.6 &    68.2 &     66.0 &       58.5 &    76.4 &       83.1 &   83.2 &   69.0 &  60.1 \\
cls\_l1 &  54.7 &    52.0 &     53.9 &       55.3 &    64.7 &       57.5 &   79.7 &   54.9 &  58.0 \\
cls\_l2 &  67.6 &    66.2 &     56.4 &       58.1 &    75.5 &       81.0 &   83.0 &   65.3 &  60.1 \\
cls\_l3 &  64.7 &    62.7 &     54.9 &       58.0 &    75.0 &       79.4 &   82.9 &   61.7 &  60.1 \\
avg\_a  &  76.0 &    78.9 &     76.4 &       75.0 &    76.5 &       85.2 &   85.6 &   78.0 &  60.1 \\
avg\_l1 &  59.4 &    57.0 &     60.6 &       47.7 &    63.0 &       57.1 &   79.2 &   56.9 &  58.0 \\
avg\_l2 &  74.3 &    77.1 &     69.1 &       61.5 &    76.4 &       83.3 &   84.9 &   74.4 &  60.4 \\
avg\_l3 &  70.9 &    76.2 &     65.2 &       55.0 &    76.5 &       80.1 &   84.5 &   70.3 &  60.5 \\
\bottomrule
\end{tabular}



<ipython-input-11-5793a97b90ac>:12: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(cur_df.to_latex(index=True))


In [ ]:
# Table in Latex to summarize the score on each property of the models
# We chose concept input, avg method, cosinus distances without LEAR, last layer
for p in range(9):
    #for method in _methods:
    print("Property",p)
    display(get_last_by_prop(_lastlayers,p)[['input','model','prop','layer',*orderedmethod]])

In [19]:
(ds_sizes[0]*sum_df['p1']+ds_sizes[1]*sum_df['p2'])/(ds_sizes[0]+ds_sizes[1])

0     46.119400
1     54.602848
2     44.165358
3     47.402232
4     44.396921
5     43.081139
6     71.872363
7     44.676751
8     28.265358
9     48.523480
10    55.782217
11    70.882833
dtype: float64

In [26]:
# Table in Latex to summarize the score on each property of the models
# We chose concept input, avg method, cosinus distances without LEAR, last layer/
# Table in paper
summary_table = []
for model in _models:
    row = [model]
    minidf = _lastlayers[_lastlayers['model']==model]
    for prop in range(9):
        score = minidf[minidf['prop']==prop]['avg_a']
        row.append(round(float(score),1))
    summary_table.append(row)
#_lastlayers[['input','model','prop','layer','cls']]
sum_df = pd.DataFrame(summary_table, columns=['model','p1','p2','p3','p4','p5','p6','p7','p8','p9'])
sum_df['model'] = ['FT-cc','FT-wiki','BERT','BERT-L','RoBERTa','RoBERTa-L','MPNet','Scibert','CTP','ERNIE','S-BERT','DistilRoB']
# sum_df['P'] = np.round( (ds_sizes[0]*sum_df['p1']+ds_sizes[1]*sum_df['p2']+ds_sizes[2]*sum_df['p3']+ds_sizes[3]*sum_df['p4'])/(ds_sizes[0]+ds_sizes[1]+ds_sizes[2]+ds_sizes[3]),1)
# sum_df['A'] = np.round( (ds_sizes[4]*sum_df['p5']+ds_sizes[5]*sum_df['p6']+ds_sizes[6]*sum_df['p7'])/(ds_sizes[4]+ds_sizes[5]+ds_sizes[6]),1)
# sum_df['S'] =np.round( (ds_sizes[7]*sum_df['p8']+ds_sizes[8]*sum_df['p9'])/(ds_sizes[7]+ds_sizes[8]),1)
sum_df['P'] = np.round( (sum_df['p1']+sum_df['p2']+sum_df['p3']+sum_df['p4'])/(4),1)
sum_df['A'] = np.round( (sum_df['p5']+sum_df['p6']+sum_df['p7'])/(3),1)
sum_df['S'] =np.round( (sum_df['p8']+sum_df['p9'])/(2),1)

sum_df

,model,p1,p2,p3,p4,p5,p6,p7,p8,p9,P,A,S
0,FT-cc,63.3,42.9,64.2,66.9,34.7,46.7,48.5,64.4,63.4,59.3,43.3,63.9
1,FT-wiki,79.7,49.9,82.0,81.1,24.1,45.9,51.0,77.4,79.0,73.2,40.3,78.2
2,BERT,76.0,38.2,72.8,70.4,22.2,41.7,44.2,75.7,73.6,64.4,36.0,74.6
3,BERT-L,78.9,41.5,77.7,77.4,24.2,45.7,50.5,76.1,75.9,68.9,40.1,76.0
4,RoBERTa,76.4,38.4,72.1,70.3,20.2,41.2,46.7,75.6,74.0,64.3,36.0,74.8
5,RoBERTa-L,75.0,37.1,72.4,70.2,23.1,36.0,37.8,75.3,73.2,63.7,32.3,74.2
6,MPNet,85.6,69.3,89.0,87.4,29.1,61.9,69.3,80.0,81.4,82.8,53.4,80.7
7,Scibert,67.5,40.4,65.8,64.1,27.2,43.0,41.7,68.7,66.4,59.4,37.3,67.6
8,CTP,60.1,22.3,40.7,31.6,23.7,24.1,21.3,65.0,57.0,38.7,23.0,61.0
9,ERNIE,78.0,43.0,74.4,74.6,23.0,42.3,47.4,75.5,75.7,67.5,37.6,75.6


In [27]:
print(sum_df.iloc[[0,1,2,3,4,5,10,11,6,9,8,7]].to_latex(index=False))

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
    model &   p1 &   p2 &   p3 &   p4 &   p5 &   p6 &   p7 &   p8 &   p9 &    P &    A &    S \\
\midrule
    FT-cc & 63.3 & 42.9 & 64.2 & 66.9 & 34.7 & 46.7 & 48.5 & 64.4 & 63.4 & 59.3 & 43.3 & 63.9 \\
  FT-wiki & 79.7 & 49.9 & 82.0 & 81.1 & 24.1 & 45.9 & 51.0 & 77.4 & 79.0 & 73.2 & 40.3 & 78.2 \\
     BERT & 76.0 & 38.2 & 72.8 & 70.4 & 22.2 & 41.7 & 44.2 & 75.7 & 73.6 & 64.4 & 36.0 & 74.6 \\
   BERT-L & 78.9 & 41.5 & 77.7 & 77.4 & 24.2 & 45.7 & 50.5 & 76.1 & 75.9 & 68.9 & 40.1 & 76.0 \\
  RoBERTa & 76.4 & 38.4 & 72.1 & 70.3 & 20.2 & 41.2 & 46.7 & 75.6 & 74.0 & 64.3 & 36.0 & 74.8 \\
RoBERTa-L & 75.0 & 37.1 & 72.4 & 70.2 & 23.1 & 36.0 & 37.8 & 75.3 & 73.2 & 63.7 & 32.3 & 74.2 \\
   S-BERT & 76.5 & 51.9 & 77.6 & 76.4 & 31.9 & 49.9 & 56.7 & 69.3 & 71.5 & 70.6 & 46.2 & 70.4 \\
DistilRoB & 85.2 & 68.2 & 89.9 & 90.8 & 33.8 & 61.2 & 68.2 & 77.8 & 79.5 & 83.5 & 54.4 & 78.6 \\
    MPNet & 85.6 & 69.3 & 89.0 & 87.4 & 29.1 & 61.9 & 69.3 & 80.0 & 81.4 & 82.

<ipython-input-27-761c2c0a06dd>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(sum_df.iloc[[0,1,2,3,4,5,10,11,6,9,8,7]].to_latex(index=False))


# Qualitative results
## Property 1

In [139]:
_models = ['bert-base-cased', 'ctp','ernie-base','sentence-transformers/bert-base-nli-mean-tokens','sentence-transformers/all-distilroberta-v1']
new_df = []
for p in range(1):
    
    for mid,model in enumerate(_models):
        path='computed_results_v2/single_results_concept_p{prop}_{model}.json'.format(prop=p,model=model.replace('/','_'))
        df = pd.read_json(load_json(path), orient="index")
        if len(new_df)==0:
            new_df = df[['tree_id','ent_1','ent_2','ent_3','def_1','def_2','def_3','valid','correct']]
            new_df.rename(columns={"correct": "correct_"+str(mid)}, inplace=True)
        else:
            new_df['correct'+str(mid)] = df['correct']


In [145]:
# Good bert, not ernie
#674    4
#701    5
#652    6
fdf = new_df[(new_df['correct_0']==True)&(new_df['correct2']==False)]
fdf[fdf['tree_id']==652] # more specific field knowledge

# ex 1
#new_df[new_df['tree_id']==652].loc[474]['def_1'],new_df[new_df['tree_id']==652].loc[474]['def_2'],new_df[new_df['tree_id']==652].loc[474]['def_3']

,tree_id,ent_1,ent_2,ent_3,def_1,def_2,def_3,valid,correct_0,correct1,correct2,correct3,correct4
484,652,black felt cup,Sarcosomataceae,ascomycete,a common name for a variety of Sarcosomataceae,a type of ascomycetous fungus,any fungus of the class Ascomycetes (or subdiv...,1,True,True,False,True,True
485,652,devil's urn,Sarcosomataceae,ascomycete,a common name for a variety of Sarcosomataceae,a type of ascomycetous fungus,any fungus of the class Ascomycetes (or subdiv...,1,True,True,False,True,True
486,652,winter urn,Sarcosomataceae,ascomycete,a common name for a variety of Sarcosomataceae,a type of ascomycetous fungus,any fungus of the class Ascomycetes (or subdiv...,1,True,True,False,True,True
487,652,charred pancake cup,Sarcosomataceae,ascomycete,a common name for a variety of Sarcosomataceae,a type of ascomycetous fungus,any fungus of the class Ascomycetes (or subdiv...,1,True,True,False,False,True
488,652,devil's cigar,Sarcosomataceae,ascomycete,a common name for a variety of Sarcosomataceae,a type of ascomycetous fungus,any fungus of the class Ascomycetes (or subdiv...,1,True,True,False,True,True
489,652,Rufous rubber cup,Sarcosomataceae,ascomycete,a common name for a variety of Sarcosomataceae,a type of ascomycetous fungus,any fungus of the class Ascomycetes (or subdiv...,1,True,True,False,True,True


In [149]:
fdf.loc[486]['def_1']

'a common name for a variety of Sarcosomataceae'

In [104]:
 # Good ernie, not bert
# Found tree 691, 14 mistakes
# 711, 7 wrong
# 697 6 wrong
new_df[(new_df['correct_0']==False)&(new_df['correct2']==True)]
fdf = new_df[(new_df['correct_0']==False)&(new_df['correct2']==True)]
fdf[fdf['tree_id']==711]


,tree_id,ent_1,ent_2,ent_3,def_1,def_2,def_3,valid,correct_0,correct1,correct2,correct3,correct4
11348,711,alarm clock,clock,timepiece,a clock that wakes a sleeper at some preset time,a timepiece that shows the time of day,a measuring instrument or device for keeping time,1,False,True,True,True,True
11351,711,pendulum clock,clock,timepiece,a clock regulated by a pendulum,a timepiece that shows the time of day,a measuring instrument or device for keeping time,1,False,True,True,True,True
11353,711,wall clock,clock,timepiece,a clock mounted on a wall,a timepiece that shows the time of day,a measuring instrument or device for keeping time,1,False,True,True,True,True
11355,711,Big Ben,clock,timepiece,clock in the clock tower of the Houses of Parl...,a timepiece that shows the time of day,a measuring instrument or device for keeping time,1,False,True,True,True,True
11357,711,chronograph,timer,timepiece,an accurate timer for recording time,a timepiece that measures a time interval and ...,a measuring instrument or device for keeping time,1,False,True,True,False,False
11358,711,parking meter,timer,timepiece,a coin-operated timer located next to a parkin...,a timepiece that measures a time interval and ...,a measuring instrument or device for keeping time,1,False,True,True,True,True
11367,711,stem-winder,watch,timepiece,a watch that is wound by turning a knob at the...,a small portable timepiece,a measuring instrument or device for keeping time,1,False,False,True,False,True


In [102]:
fdf.loc[8054]['def_1'],fdf.loc[8054]['def_2'],fdf.loc[8054]['def_3'] # outrage, atrocity, cruelty, Similar portion phrases 'cruel act'

('a wantonly cruel act',
 'an act of atrocious cruelty',
 'a cruel act; a deliberate infliction of pain and suffering')

In [103]:
fdf.loc[8057]['def_1'],fdf.loc[8057]['def_2'],fdf.loc[8057]['def_3'] # Crucifixion, torture, persecution

('the infliction of extremely painful punishment or suffering',
 'the deliberate, systematic, or wanton infliction of physical or mental suffering by one or more persons in an attempt to force another person to yield information or to make a confession or for any other reason',
 'the act of persecuting (especially on the basis of race or religion)')

In [105]:
fdf.loc[11348]['def_1'],fdf.loc[11348]['def_2'],fdf.loc[11348]['def_3'] # alarm clock, clokc, timeplace, Similar portion phrases 'cruel act'

('a clock that wakes a sleeper at some preset time',
 'a timepiece that shows the time of day',
 'a measuring instrument or device for keeping time')

In [115]:
# Good ernie, s-bert but not bert
# Bigest subtrees
# 711     5
# 741     5
# 691    13
fdf = new_df[(new_df['correct_0']==False)&(new_df['correct2']==True)&(new_df['correct3']==True)]
fdf.head()
#fdf.groupby('tree_id').count()['valid'].sort_values() # similar to ernie

,tree_id,ent_1,ent_2,ent_3,def_1,def_2,def_3,valid,correct_0,correct1,correct2,correct3,correct4
112,649,coextension,extension,longness,equality of extension or duration,amount or degree or range to which something e...,the property of being of long spatial extent,1,False,False,True,True,True
336,651,pickelhaube,helmet,armor plate,a spiked helmet worn by German soldiers,armor plate that protects the head,specially hardened steel plate used to protect...,1,False,False,True,True,True
697,653,tennis lesson,lesson,teaching,a lesson in playing tennis,a unit of instruction,the profession of a teacher,1,False,False,True,True,False
1843,659,vox humana,reed stop,organ stop,an organ reed stop producing tones imitative o...,an organ stop with the tone of a reed instrument,a graduated set of organ pipes of like tone qu...,1,False,False,True,True,False
2231,663,Manchester terrier,rat terrier,terrier,a breed of short-haired rat terrier with a bla...,any of several breeds of terrier developed to ...,any of several usually small short-bodied bree...,1,False,True,True,True,True


In [119]:
#Corect sbert not ernie 
#701     5
# 652    11
# 673    15
#fdf.groupby('tree_id').count()['valid'].sort_values() # similar to ernie
fdf = new_df[(new_df['correct2']==False)&(new_df['correct3']==True)]
#fdf.groupby('tree_id').count()['valid'].sort_values() 
fdf[fdf['tree_id']==673] # diff major/mino tranquilizer, tranquilizer

,tree_id,ent_1,ent_2,ent_3,def_1,def_2,def_3,valid,correct_0,correct1,correct2,correct3,correct4
4511,673,chlorpromazine,major tranquilizer,tranquilizer,a drug (trade name Thorazine) derived from phe...,tranquilizer used to treat psychotic condition...,a drug used to reduce stress or tension withou...,1,False,False,False,True,False
4512,673,prochlorperazine,major tranquilizer,tranquilizer,antipsychotic and antiemetic drug used to trea...,tranquilizer used to treat psychotic condition...,a drug used to reduce stress or tension withou...,1,False,True,False,True,True
4514,673,molindone,major tranquilizer,tranquilizer,antipsychotic drug (trade name Moban) used in ...,tranquilizer used to treat psychotic condition...,a drug used to reduce stress or tension withou...,1,False,True,False,True,True
4515,673,thioridazine,major tranquilizer,tranquilizer,a tranquilizer (trade name Mellaril) used to t...,tranquilizer used to treat psychotic condition...,a drug used to reduce stress or tension withou...,1,True,True,False,True,True
4516,673,diphenylbutyl piperidine,major tranquilizer,tranquilizer,a group of antipsychotic drugs used mainly in ...,tranquilizer used to treat psychotic condition...,a drug used to reduce stress or tension withou...,1,False,True,False,True,True
4517,673,loxapine,major tranquilizer,tranquilizer,a tranquilizer (trade name Loxitane) used to t...,tranquilizer used to treat psychotic condition...,a drug used to reduce stress or tension withou...,1,False,True,False,True,True
4518,673,lithium carbonate,major tranquilizer,tranquilizer,a white powder (LiCO3) used in manufacturing g...,tranquilizer used to treat psychotic condition...,a drug used to reduce stress or tension withou...,1,False,True,False,True,True
4521,673,thiothixene,major tranquilizer,tranquilizer,a tranquilizer (trade name Navane) used to tre...,tranquilizer used to treat psychotic condition...,a drug used to reduce stress or tension withou...,1,False,True,False,True,False
4522,673,perphenazine,minor tranquilizer,tranquilizer,tranquilizer and antidepressant (trade name Tr...,a tranquilizer used to relieve anxiety and red...,a drug used to reduce stress or tension withou...,1,False,True,False,True,False
4523,673,benzodiazepine,minor tranquilizer,tranquilizer,any of several similar lipophilic amines used ...,a tranquilizer used to relieve anxiety and red...,a drug used to reduce stress or tension withou...,1,True,False,False,True,False


In [120]:
pos = 4516
fdf.loc[pos]['def_1'],fdf.loc[pos]['def_2'],fdf.loc[pos]['def_3'] # outrage, atrocity, cruelty, Similar portion phrases 'cruel act'

('a group of antipsychotic drugs used mainly in the treatment of schizophrenia',
 'tranquilizer used to treat psychotic conditions when a calming effect is desired',
 'a drug used to reduce stress or tension without reducing mental clarity')

In [121]:
fdf[fdf['tree_id']==652] # diff major/mino tranquilizer, tranquilizer

,tree_id,ent_1,ent_2,ent_3,def_1,def_2,def_3,valid,correct_0,correct1,correct2,correct3,correct4
475,652,Sarcoscypha coccinea,discomycete,ascomycete,a discomycete that is a harbinger of spring; t...,any fungus that is a member of the subclass Di...,any fungus of the class Ascomycetes (or subdiv...,1,False,False,False,True,True
476,652,blood cup,discomycete,ascomycete,a scarlet European fungus with cup-shaped asco...,any fungus that is a member of the subclass Di...,any fungus of the class Ascomycetes (or subdiv...,1,False,False,False,True,False
478,652,Leotia lubrica,discomycete,ascomycete,a discomycete that develops in clusters of sli...,any fungus that is a member of the subclass Di...,any fungus of the class Ascomycetes (or subdiv...,1,False,False,False,True,True
479,652,Peziza domicilina,discomycete,ascomycete,a discomycetous fungus of the genus Peziza; th...,any fungus that is a member of the subclass Di...,any fungus of the class Ascomycetes (or subdiv...,1,False,False,False,True,True
480,652,Aleuria aurantia,discomycete,ascomycete,a discomycete with bright orange cup-shaped or...,any fungus that is a member of the subclass Di...,any fungus of the class Ascomycetes (or subdiv...,1,False,False,False,True,False
482,652,Urnula craterium,discomycete,ascomycete,an urn-shaped discomycete with a nearly black ...,any fungus that is a member of the subclass Di...,any fungus of the class Ascomycetes (or subdiv...,1,False,False,False,True,False
484,652,black felt cup,Sarcosomataceae,ascomycete,a common name for a variety of Sarcosomataceae,a type of ascomycetous fungus,any fungus of the class Ascomycetes (or subdiv...,1,True,True,False,True,True
485,652,devil's urn,Sarcosomataceae,ascomycete,a common name for a variety of Sarcosomataceae,a type of ascomycetous fungus,any fungus of the class Ascomycetes (or subdiv...,1,True,True,False,True,True
486,652,winter urn,Sarcosomataceae,ascomycete,a common name for a variety of Sarcosomataceae,a type of ascomycetous fungus,any fungus of the class Ascomycetes (or subdiv...,1,True,True,False,True,True
488,652,devil's cigar,Sarcosomataceae,ascomycete,a common name for a variety of Sarcosomataceae,a type of ascomycetous fungus,any fungus of the class Ascomycetes (or subdiv...,1,True,True,False,True,True


In [122]:
pos = 478
fdf.loc[pos]['def_1'],fdf.loc[pos]['def_2'],fdf.loc[pos]['def_3'] # takes out better ifnromation (specialized)

('a discomycete that develops in clusters of slippery rubbery gelatinous fruiting bodies that are dingy yellow to tan in color',
 'any fungus that is a member of the subclass Discomycetes',
 'any fungus of the class Ascomycetes (or subdivision Ascomycota) in which the spores are formed inside an ascus')

## Property 2

In [123]:
_models = ['bert-base-cased', 'ctp','ernie-base','sentence-transformers/bert-base-nli-mean-tokens','sentence-transformers/all-distilroberta-v1']
new_df = []
for p in range(1,2):
    
    for mid,model in enumerate(_models):
        path='computed_results_v2/single_results_concept_p{prop}_{model}.json'.format(prop=p,model=model.replace('/','_'))
        df = pd.read_json(load_json(path), orient="index")
        if len(new_df)==0:
            new_df = df[['tree_id','ent_1','ent_2','ent_3','def_1','def_2','def_3','valid','correct']]
            new_df.rename(columns={"correct": "correct_"+str(mid)}, inplace=True)
        else:
            new_df['correct'+str(mid)] = df['correct']

In [130]:
# Good bert, not ernie
fdf = new_df[(new_df['correct_0']==True)&(new_df['correct2']==False)]
#fdf.groupby('tree_id').count()['valid'].sort_values()
# Top subtrees
# 741    31
# 703    33
# 757    53
fdf[fdf['tree_id']==757] # more specific field knowledge
pos = 84876
fdf.loc[pos]['def_1'],fdf.loc[pos]['def_2'],fdf.loc[pos]['def_3'] # takes out better ifnromation (specialized)

('thick-walled asexual resting spore of certain fungi and algae',
 'a small usually single-celled asexual reproductive body produced by many nonflowering plants and fungi and some bacteria and protozoans and that are capable of developing into a new individual without sexual fusion',
 'one of the four asexual spores produced within a sporangium')

In [133]:
fdf[fdf['tree_id']==703] # more specific field knowledge
pos = 41738
fdf.loc[pos]['def_1'],fdf.loc[pos]['def_2'],fdf.loc[pos]['def_3'] # takes out better ifnromation (specialized)

('made with three slices of usually toasted bread',
 'two (or more) slices of bread with a filling between them',
 'sandwich without a covering slice of bread')

In [135]:
# Good ernie, not bert
fdf = new_df[(new_df['correct_0']==False)&(new_df['correct2']==True)]
fdf.groupby('tree_id').count()['valid'].sort_values()
# Top trees
# 686     46
# 691    111
# 741    130

tree_id
649      1
709      1
700      1
752      1
755      1
      ... 
730     41
758     43
686     46
691    111
741    130
Name: valid, Length: 83, dtype: int64

In [137]:
fdf[fdf['tree_id']==741].loc[70232]

tree_id                                                    741
ent_1                                                     agar
ent_2                                                      gum
ent_3                                                gum butea
def_1        a colloidal extract of algae; used especially ...
def_2        any of various substances (soluble in water) t...
def_3        dried juice of the dhak tree; used as an astri...
valid                                                        1
correct_0                                                False
correct1                                                  True
correct2                                                  True
correct3                                                  True
correct4                                                  True
Name: 70232, dtype: object

In [138]:
fdf[fdf['tree_id']==741].loc[70232]['def_1'],fdf[fdf['tree_id']==741].loc[70232]['def_2'],fdf[fdf['tree_id']==741].loc[70232]['def_3']
# traits on 'algae' - 'tree'. 'algae' - 'plants', 'food' ' juice'

('a colloidal extract of algae; used especially in culture media and as a gelling agent in foods',
 'any of various substances (soluble in water) that exude from certain plants; they are gelatinous when moist but harden on drying',
 'dried juice of the dhak tree; used as an astringent')

# Gather results obtained after parallel launch with gather_results.py script (Hyponymy LEAR)

In [8]:
basepath = './computed_results_hypo'
_dataset_base = "/users/cost/jlovonme/data"
_inputtype = 'concept' # name or concept
#_methods = ['cls','avg_all']
_methods = ['cls','avg_a','cls_l1','avg_l1','cls_l2','avg_l2','cls_l3','avg_l3','ecls','eavg','ecls_l1','eavg_l1','ecls_l2','eavg_l2','ecls_l3','eavg_l3']
_models = ['ft-cc','ft-wiki','bert-base-cased','bert-large-cased','roberta-base','roberta-large','sentence-transformers/all-mpnet-base-v2',
          #'allenai/scibert_scivocab_cased','ctp',
           'ernie-base','sentence-transformers/bert-base-nli-mean-tokens','sentence-transformers/all-distilroberta-v1']
#_models = ['bert-base-cased']
orderedmethod = ['cls','cls_l1','cls_l2','cls_l3','avg_a','avg_l1','avg_l2','avg_l3','ecls','ecls_l1','ecls_l2','ecls_l3','eavg','eavg_l1','eavg_l2','eavg_l3']

In [9]:
_bigdf = create_table_results('concept', _models,props=9 )
_lastlayers = get_last_layers(_bigdf)

In [10]:
for p in range(1):
    #for method in _methods:
    print("Property",p)
    display(get_last_by_prop(_lastlayers,p)[['input','model','prop','layer',*orderedmethod]])

Property 0


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
0,concept,bert-base-cased,0,12,72.61,45.78,61.95,56.24,76.03,38.68,67.94,57.35,72.40,72.47,72.40,72.61,75.68,75.89,75.61,75.68
0,concept,bert-large-cased,0,24,68.15,53.24,64.95,63.48,78.89,50.38,77.42,72.61,68.71,68.57,68.78,68.78,77.98,78.12,77.98,77.98
0,concept,ernie-base,0,12,68.99,40.63,57.63,50.73,78.05,40.21,73.38,62.58,69.20,68.85,69.06,68.99,77.77,77.70,77.70,77.70
0,concept,ft-cc,0,0,63.34,52.33,55.40,52.75,63.34,52.33,55.40,52.75,60.35,57.00,59.23,57.21,60.35,57.00,59.23,57.21
0,concept,ft-wiki,0,0,79.65,53.10,72.13,62.02,79.65,53.10,72.13,62.02,77.00,78.47,77.42,78.19,77.00,78.47,77.42,78.19
0,concept,roberta-base,0,12,65.99,49.55,51.15,50.45,76.38,37.77,44.25,40.49,66.27,61.18,66.27,65.92,76.59,76.31,76.52,76.52
0,concept,roberta-large,0,24,58.54,55.05,58.75,59.16,75.05,46.62,56.45,50.52,58.54,58.54,58.54,58.54,74.84,74.08,74.84,74.84
0,concept,sentence-transformers/all-distilroberta-v1,0,6,83.14,52.20,82.72,80.70,85.16,49.13,79.86,74.15,79.09,79.30,79.09,79.09,80.63,81.05,80.63,80.70
0,concept,sentence-transformers/all-mpnet-base-v2,0,12,83.21,77.70,82.86,82.09,85.64,76.03,85.78,84.39,83.14,83.28,83.28,83.41,85.64,85.71,85.71,85.71
0,concept,sentence-transformers/bert-base-nli-mean-tokens,0,12,76.45,52.89,74.91,73.73,76.52,46.62,75.75,73.94,76.59,76.72,76.59,76.59,77.28,77.35,77.28,77.28


In [12]:
basepath = './computed_results_hyper'
_bigdf = create_table_results('concept', _models,props=1 )
_lastlayers = get_last_layers(_bigdf)

In [14]:
for p in range(1):
    #for method in _methods:
    print("Property",p)
    df = get_last_by_prop(_lastlayers,p)[['input','model','prop','layer',*orderedmethod]]
    display(get_last_by_prop(_lastlayers,p)[['input','model','prop','layer',*orderedmethod]])

Property 0


,input,model,prop,layer,cls,cls_l1,cls_l2,cls_l3,avg_a,avg_l1,avg_l2,avg_l3,ecls,ecls_l1,ecls_l2,ecls_l3,eavg,eavg_l1,eavg_l2,eavg_l3
0,concept,bert-base-cased,0,12,72.61,56.45,68.01,63.97,76.03,63.69,74.49,72.89,72.40,72.20,72.40,72.47,75.68,75.61,75.68,75.68
0,concept,bert-large-cased,0,24,68.15,48.01,68.43,63.14,78.89,54.15,75.75,72.54,68.71,68.92,68.71,68.85,77.98,77.63,77.91,77.91
0,concept,ernie-base,0,12,68.99,61.39,71.15,67.94,78.05,64.39,76.59,73.03,69.20,69.62,69.13,69.13,77.77,78.05,77.70,77.70
0,concept,ft-cc,0,0,63.34,55.82,57.63,55.68,63.34,55.82,57.63,55.68,60.35,56.59,59.65,56.86,60.35,56.59,59.65,56.86
0,concept,ft-wiki,0,0,79.65,62.23,72.68,68.78,79.65,62.23,72.68,68.78,77.00,73.45,76.52,75.26,77.00,73.45,76.52,75.26
0,concept,roberta-base,0,12,65.99,50.45,52.20,50.52,76.38,63.00,69.83,67.39,66.27,62.44,66.34,66.90,76.59,76.45,76.52,76.38
0,concept,roberta-large,0,24,58.54,47.94,58.12,58.05,75.05,53.45,65.51,59.51,58.54,58.54,58.54,58.54,74.84,75.05,74.84,74.84
0,concept,sentence-transformers/all-distilroberta-v1,0,6,83.14,54.63,79.51,77.56,85.16,56.72,81.46,77.49,79.09,79.09,79.09,79.09,80.63,80.07,80.56,80.49
0,concept,sentence-transformers/all-mpnet-base-v2,0,12,83.21,77.28,83.34,82.23,85.64,77.98,85.78,85.85,83.14,82.93,83.14,83.34,85.64,85.78,85.71,85.92
0,concept,sentence-transformers/bert-base-nli-mean-tokens,0,12,76.45,60.56,76.45,75.75,76.52,64.53,77.42,78.05,76.59,76.66,76.59,76.59,77.28,77.21,77.28,77.28


In [23]:
df[['model','cls','avg_a','ecls','eavg']].set_index('model').transpose()
#df[['model','cls','avg_a','ecls','eavg']].transpose()

model,bert-base-cased,bert-large-cased,ernie-base,ft-cc,ft-wiki,roberta-base,roberta-large,sentence-transformers/all-distilroberta-v1,sentence-transformers/all-mpnet-base-v2,sentence-transformers/bert-base-nli-mean-tokens
cls,72.61,68.15,68.99,63.34,79.65,65.99,58.54,83.14,83.21,76.45
avg_a,76.03,78.89,78.05,63.34,79.65,76.38,75.05,85.16,85.64,76.52
ecls,72.40,68.71,69.20,60.35,77.00,66.27,58.54,79.09,83.14,76.59
eavg,75.68,77.98,77.77,60.35,77.00,76.59,74.84,80.63,85.64,77.28


In [29]:
# table to show in Latex: We study only property 1 because similar trends in others\
# Also main works focus on hypernym, so we study representations on property 1 that fit best to this type of evaluation, to be fair with enhanced models
# We use the concept input (w/ definitions) because they are contextualized models
# We evalaute cosinus distance, since euclidean gives similar trends, and it's a lot of space
cur_df = get_last_by_prop(_lastlayers,0)[['model','cls','avg_a','ecls','eavg']]
for m in ['cls','avg_a','ecls','eavg']:
    cur_df[m]=cur_df[m].round(decimals=1)
cur_df['model'] = ['BERT','BERT-L','ERNIE','FT-cc','FT-wiki','RoBERTa','RoBERTa-L','DistilRoB','MPNet','S-BERT']
cur_df = cur_df.set_index('model')
cur_df = cur_df.transpose()[['BERT','BERT-L','RoBERTa','RoBERTa-L','S-BERT','DistilRoB','MPNet','ERNIE']]
#cur_df = cur_df.transpose()[]
print(cur_df.to_latex(index=True))

\begin{tabular}{lrrrrrrrr}
\toprule
model &  BERT &  BERT-L &  RoBERTa &  RoBERTa-L &  S-BERT &  DistilRoB &  MPNet &  ERNIE \\
\midrule
cls   &  72.6 &    68.2 &     66.0 &       58.5 &    76.4 &       83.1 &   83.2 &   69.0 \\
avg\_a &  76.0 &    78.9 &     76.4 &       75.0 &    76.5 &       85.2 &   85.6 &   78.0 \\
ecls  &  72.4 &    68.7 &     66.3 &       58.5 &    76.6 &       79.1 &   83.1 &   69.2 \\
eavg  &  75.7 &    78.0 &     76.6 &       74.8 &    77.3 &       80.6 &   85.6 &   77.8 \\
\bottomrule
\end{tabular}



<ipython-input-29-487907b8edbc>:12: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(cur_df.to_latex(index=True))
